# El tratamiendo de las variables categóricas

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
data_frame = pd.read_csv("../datasets/ecom-expense/Ecom Expense.csv")
data_frame.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


In [3]:
# vemos que hay dos variables categóricas, GENDER y CITY TIER, toca crear dummies para volverlas numéricas. 
dummy_gender = pd.get_dummies(data_frame["Gender"], prefix="Gender")
dummy_city_tier = pd.get_dummies(data_frame["City Tier"], prefix="City")

In [4]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [5]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [6]:
#Tenemos qu hacer un join de estos datasets que categorizan con el dataframe original
column_names = data_frame.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [7]:
data_frame_new = data_frame[column_names].join(dummy_gender)
column_names = data_frame_new.columns.values.tolist()
data_frame_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


In [9]:
data_frame_new = data_frame_new[column_names].join(dummy_city_tier)
data_frame_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


In [21]:
# Ahora vamos a reducir el dataset a lo que nos importa:
feature_cols = ["Monthly Income","Transaction Time",
               "Gender_Female","Gender_Male",
               "City_Tier 1","City_Tier 2","City_Tier 3", 
               "Record"]

In [22]:
var_entrada = data_frame_new[feature_cols]
var_salida = data_frame["Total Spend"]

In [23]:
linear_regression = LinearRegression()
linear_regression.fit(var_entrada, var_salida)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [24]:
print(linear_regression.intercept_)
print(linear_regression.coef_)

-79.41713030136816
[ 1.47538980e-01  1.54946125e-01 -1.31025013e+02  1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]


In [25]:
# podemos juntar cada una de las columnas con sus coeficientes con ZIP!
list(zip(feature_cols, linear_regression.coef_))

[('Monthly Income', 0.14753898049205733),
 ('Transaction Time', 0.1549461254958966),
 ('Gender_Female', -131.0250132555464),
 ('Gender_Male', 131.02501325554658),
 ('City_Tier 1', 76.76432601049548),
 ('City_Tier 2', 55.138974309232275),
 ('City_Tier 3', -131.9033003197278),
 ('Record', 772.2334457445638)]

Obserar como en los dummies los valores se complementan de manera que si se suman entre los que se relacionan dan como resultado 0. 


In [26]:
# Si queremos ver que tan bueno es el modelo:  SI SCORE es baja, toca empezar a añadir DE A UNA de las otras var.

linear_regression.score(var_entrada, var_salida)

0.9179923586131016

* **Se agregará RECORD en el data set seleccionado para ver si cambia! entonces retroceder a donde se escogieron las variables que serían las de entrada para el modelo y revisar si el escore empieza a subir**

* **Después se agregó AGE y nos dimos cuenta que no sirve, no es relavante**

Después de lo anterior el modelo puede ser descrito entonces como: 

* **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+
 ('Transaction Time'* 0.1549461254958966)+
 ('Gender_Female'* -131.0250132555464)+
 ('Gender_Male'* 131.02501325554658)+
 ('City_Tier 1'* 76.76432601049548)+
 ('City_Tier 2'* 55.138974309232275)+
 ('City_Tier 3'* -131.9033003197278)+
 ('Record'* 772.2334457445638)
     
     * **Si es hombre y vive en CT1:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)+ 131.02501325554658+ 76.76432601049548 + ('Record'* 772.2334457445638)
     * **Si es hombre y vive en CT2:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)+ 131.02501325554658+ 55.138974309232275 + ('Record'* 772.2334457445638)
     * **Si es hombre y vive en CT3:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)+ 131.02501325554658 -131.9033003197278 + ('Record'* 772.2334457445638)
     * **Si es mujer y vive en CT1:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)- 131.02501325554658+ 76.76432601049548 + ('Record'* 772.2334457445638)
     * **Si es mujer y vive en CT2:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)- 131.02501325554658+ 55.138974309232275 + ('Record'* 772.2334457445638)
     * **Si es mujer y vive en CT3:** 
         * **TotalSpend** = -79.41713030136816 + ('Monthly Income' * 0.14753898049205733)+('Transaction Time'* 0.1549461254958966)- 131.02501325554658 -131.9033003197278 + ('Record'* 772.2334457445638)
 
 Vamos entonces a recrear la predicción en una nueva columna del dataset seleccionado:


In [48]:
data_frame_new["prediction"] = -79.41713030136816 + (data_frame_new['Monthly Income'] * 0.14753898049205733)+(data_frame_new['Transaction Time'] * 0.1549461254958966)+(data_frame_new['Gender_Female'] * -131.0250132555464)+(data_frame_new['Gender_Male'] * 131.02501325554658)+(data_frame_new['City_Tier 1'] * 76.76432601049548)+(data_frame_new['City_Tier 2'] * 55.138974309232275)+(data_frame_new['City_Tier 3'] * -131.9033003197278)+(data_frame_new['Record'] * 772.2334457445638)

In [49]:
data_frame_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


In [50]:
SSD = np.sum((data_frame_new["prediction"]-data_frame_new["Total Spend"])**2)
SSD

1517733985.340816

In [51]:
# 803 euros arriba y 803 euros abajo!
RSE = np.sqrt(SSD/(len(data_frame_new)-len(feature_cols)-1))
RSE

803.1318809818165

In [52]:
total_spend_mean = np.mean(data_frame_new["Total Spend"])
total_spend_mean

6163.176415976714

In [53]:
# Error de 13% !
error = RSE/total_spend_mean
error

0.13031135680294162

In [54]:
# Otra forma de hacerlo más fácil es a través de PREDICTION 
data_frame_new["new_prediction"] = linear_regression.predict(pd.DataFrame(data_frame_new[feature_cols]))

In [56]:
# Vemos que son exactamente los mismos valores sin tener necesidad de copiar TODO el modelo. 
data_frame_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction,new_prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335,3581.980335
